# Revocation - Bob

## Role: Credential Holder

There will be two parts to this notebook:

- Part 1: Alice issues a revocable credential to [Bob](http://localhost:8889/lab/tree/5%20Advanced%20Concepts/Part%202%20-%20Revocation.ipynb). Bob will store this credential and present proof of it back to Alice while it is not revoked, and Alice will verify the presentation.
- Part 2: Alice then revokes the credential issued to Bob in Part 1 and requests another proof from Bob. This second presentation from Bob will fail to verify.

For details on how revocation works on Hyperledger Indy, you can read more [here](https://github.com/hyperledger/indy-hipe/tree/master/text/0011-cred-revocation).

# Part 1 - Issuing & Proving Non-Revoked Credential

## Complete steps 1-6 in [Alice's revocation notebook](http://localhost:8888/lab/tree/5%20Advanced%20Concepts/Part%202%20-%20Revocation.ipynb)

## 7. Initialise Bob's controller

In [1]:
%autoawait
import time
import asyncio

from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = ""
ADMIN_URL = "http://bob-agent:8051"

agent_controller = AriesAgentController(admin_url=ADMIN_URL)

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

## 8. Register listeners

The handler should get called every time the controller receives a webhook with the topic `issue_credential` and `present_proof`, printing out the payload. The agent calls to this webhook every time it receives protocol messages for these topics from a credential.

In [3]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

<Task pending name='Task-4' coro=<AriesAgentController.listen_webhooks() running at /aries_basic_controller/aries_controller.py:61>>

In [4]:
def cred_handler(payload):
    print("Handle Credentials")
    global connection_id
    connection_id = payload['connection_id']
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Offering: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}

def proof_handler(payload):
    print("Handle present proof")
    role = payload["role"]
    global pres_ex_id
    pres_ex_id = payload["presentation_exchange_id"]
    state = payload["state"]
    print(f"Role {role}, Exchange {pres_ex_id} in state {state}")

proof_listener = {
    "topic": "present_proof",
    "handler": proof_handler
}

agent_controller.register_listeners([cred_listener, proof_listener], defaults=True)

Subscribing too: issue_credential
Subscribing too: present_proof
Handle Credentials
Credential exchange 724cb68a-4c5c-4e75-8e94-0ebbefceaadf, role: holder, state: offer_received
Offering: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}, {'name': 'age', 'value': '21'}]


## 9. Continue to [Alice's Notebook](http://localhost:8888/lab/tree/5%20Advanced%20Concepts/Part%202%20-%20Revocation.ipynb)

Alice will issue Bob a revocable credential.

## 10. Check your credential records

You should have one recording the credential offer from Alice. This should also have shown up as as a print statement from the handler function.

In [5]:
response = await agent_controller.issuer.get_records()
results = response["results"]
print(len(results))
if len(results) == 0:
    print("You need to first send a credential from the issuer notebook (Alice)")
else:
    cred_record = results[0]
    cred_ex_id = cred_record['credential_exchange_id']
    state = cred_record['state']
    role = cred_record['role']
    attributes = results[0]['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")
    print(f"Being offered: {attributes}")

1
Credential exchange 724cb68a-4c5c-4e75-8e94-0ebbefceaadf, role: holder, state: offer_received
Being offered: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}, {'name': 'age', 'value': '21'}]


## 11. Request credential from Alice

If happy with the attributes being offered in the credential, then the holder requests the credential from the issuer to proceed with the issuance.

It is only possible to request a credential from an exchange when it is in the `offer_received` state.

In [6]:
record = await agent_controller.issuer.send_request_for_record(cred_ex_id)
state = record['state']
role = record['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Handle Credentials
Credential exchange 724cb68a-4c5c-4e75-8e94-0ebbefceaadf, role: holder, state: request_sent
Offering: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}, {'name': 'age', 'value': '21'}]
Credential exchange 724cb68a-4c5c-4e75-8e94-0ebbefceaadf, role: holder, state: request_sent
Handle Credentials
Credential exchange 724cb68a-4c5c-4e75-8e94-0ebbefceaadf, role: holder, state: credential_received
Offering: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}, {'name': 'age', 'value': '21'}]


## 12. Store the redential

Once the issuer has responded to a request by sending the credential, the holder needs to store it to save the credential for later.

First check that the credential record is in the `credential_received` state.

In [7]:
# This can be anything you want. You will use it later to fetch this credential from the agent storage.
credential_id = "My Revocable Credential"

In [8]:
response = await agent_controller.issuer.store_credential(cred_ex_id, credential_id)
state = response['state']
role = response['role']
print(f"Credential exchange {cred_ex_id}, role: {role}, state: {state}")

Handle Credentials
Credential exchange 724cb68a-4c5c-4e75-8e94-0ebbefceaadf, role: holder, state: credential_acked
Offering: [{'name': 'name', 'value': 'Bob'}, {'name': 'skill', 'value': 'researcher'}, {'name': 'age', 'value': '21'}]
Credential exchange 724cb68a-4c5c-4e75-8e94-0ebbefceaadf, role: holder, state: credential_acked


## 13. Check revocation status of the credential

It should not be revoked.

In [9]:
response = await agent_controller.credentials.is_revoked(credential_id)
response

{'revoked': False}

Handle present proof
Role prover, Exchange 2a6aed06-4b92-445c-b002-a26adaf52d29 in state request_received


## 14. Continue in [Alice's Notebook](http://localhost:8888/lab/tree/5%20Advanced%20Concepts/Part%202%20-%20Revocation.ipynb)

Send a presentation request for a non revoked credential proof.


## 15. Fetch redentials for presentation

Note `pres_ex_id` is set in the proofs handler.

TODO: make these steps clearer.

In [10]:
pres_ex_id
response = await agent_controller.proofs.get_record_by_id(pres_ex_id)
presentation_request = response['presentation_request']
presentation_request

{'name': 'Proof of Personal Information',
 'version': '1.0',
 'requested_attributes': {'0_skill_uuid': {'name': 'skill',
   'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}],
   'non_revoked': {'to': 1672626954}}},
 'requested_predicates': {},
 'non_revoked': {'to': 1672626955},
 'nonce': '771164708509754770507516'}

In [18]:
if state == "request_received":
    print(
    "Received Request -> Query for credentials in the wallet that satisfy the proof request")
    
if presentation_request != None:
    # include self-attested attributes (not included in credentials)
    credentials_by_reft = {}
    revealed = {}
    self_attested = {}
    predicates = {}

    # select credentials to provide for the proof
    credentials = await agent_controller.proofs.get_presentation_credentials(pres_ex_id)
    credentials = credentials[1:]
    print(credentials)

    if credentials:
        for row in sorted(
        credentials,
        key=lambda c: dict(c["cred_info"]["attrs"]),
        reverse=True,
        ):
       
            for referent in row["presentation_referents"]:
                if referent not in credentials_by_reft:
                    credentials_by_reft[referent] = row

    for referent in presentation_request["requested_attributes"]:
        if referent in credentials_by_reft:
            revealed[referent] = {
                "cred_id": credentials_by_reft[referent]["cred_info"][
                    "referent"
                ],
                "revealed": True,
            }

        else:
            self_attested[referent] = "South Africa"

    for referent in presentation_request["requested_predicates"]:
        if referent in credentials_by_reft:
            predicates[referent] = {
                "cred_id": credentials_by_reft[referent]["cred_info"][
                    "referent"
                ]
            }

    print("\nGenerate the proof")
    proof = {
        "requested_predicates": predicates,
        "requested_attributes": revealed,
        "self_attested_attributes": self_attested,
    }
    print(proof)
    print("\nXXX")
    print("\npredicates:\n{}".format(predicates))
    print("\nrevealed:\n{}".format(revealed))
    print("\nself_attested:\n{}".format(self_attested))
    
else: 
    print("No presenation record identifier")

[{'cred_info': {'referent': 'My Revocable Credential', 'attrs': {'skill': 'researcher', 'age': '21', 'name': 'Bob'}, 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:test_revocable_schema:0.0.1', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:20:default', 'rev_reg_id': 'PQRXDxdGqQGSZ8z69p4xZP:4:PQRXDxdGqQGSZ8z69p4xZP:3:CL:20:default:CL_ACCUM:f0bdd25d-4fe5-4edc-858a-293a6aa2cb54', 'cred_rev_id': '1'}, 'interval': {'from': None, 'to': 1672626954}, 'presentation_referents': ['0_skill_uuid']}]

Generate the proof
{'requested_predicates': {}, 'requested_attributes': {'0_skill_uuid': {'cred_id': 'My Revocable Credential', 'revealed': True}}, 'self_attested_attributes': {}}

XXX

predicates:
{}

revealed:
{'0_skill_uuid': {'cred_id': 'My Revocable Credential', 'revealed': True}}

self_attested:
{}


## 16. Send presentation

Bob sends the proof presentation to Alice for verification.

In [19]:
response = await agent_controller.proofs.send_presentation(pres_ex_id, proof)
print(response)

{'trace': False, 'thread_id': 'd811a59a-fe4f-41f1-bf85-24c6ae15c11b', 'role': 'prover', 'presentation_request_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/request-presentation', '@id': 'd811a59a-fe4f-41f1-bf85-24c6ae15c11b', 'request_presentations~attach': [{'@id': 'libindy-request-presentation-0', 'mime-type': 'application/json', 'data': {'base64': 'eyJuYW1lIjogIlByb29mIG9mIFBlcnNvbmFsIEluZm9ybWF0aW9uIiwgInZlcnNpb24iOiAiMS4wIiwgInJlcXVlc3RlZF9hdHRyaWJ1dGVzIjogeyIwX3NraWxsX3V1aWQiOiB7Im5hbWUiOiAic2tpbGwiLCAicmVzdHJpY3Rpb25zIjogW3siaXNzdWVyX2RpZCI6ICJQUVJYRHhkR3FRR1NaOHo2OXA0eFpQIn1dLCAibm9uX3Jldm9rZWQiOiB7InRvIjogMTY3MjYyNjk1NH19fSwgInJlcXVlc3RlZF9wcmVkaWNhdGVzIjoge30sICJub25fcmV2b2tlZCI6IHsidG8iOiAxNjcyNjI2OTU1fSwgIm5vbmNlIjogIjc3MTE2NDcwODUwOTc1NDc3MDUwNzUxNiJ9'}}]}, 'connection_id': '9777a4c5-00c9-4816-acd3-3139431cee8b', 'updated_at': '2023-01-02 02:43:15.056477Z', 'initiator': 'external', 'state': 'presentation_sent', 'presentation_exchange_id': '2a6aed0

## 17. Continue in [Alice's Notebook](http://localhost:8888/lab/tree/5%20Advanced%20Concepts/Part%202%20-%20Revocation.ipynb)

Alice will verify the non-revoked credential to be OK.

# Part 2 - Revoking the credential

## 19. Check credential

Bob will check his credential now to find that it is revoked.

In [20]:
response = await agent_controller.credentials.is_revoked(credential_id)
response

{'revoked': True}

Handle present proof
Role prover, Exchange 3185a034-0ba0-42c2-bc1e-1ceebfd99305 in state request_received


## 20. Continue in [Alice's Notebook](http://localhost:8888/lab/tree/5%20Advanced%20Concepts/Part%202%20-%20Revocation.ipynb)

Alice will request for a credential proof once again.

## 21. Fetch credentials for presentation proof

Note `pres_ex_id` is set in the proofs handler.

In [21]:
# fetch credentials for presenting proof
pres_ex_id
response = await agent_controller.proofs.get_record_by_id(pres_ex_id)
presentation_request = response['presentation_request']
presentation_request

{'name': 'Proof of Personal Information',
 'version': '1.0',
 'requested_attributes': {'0_skill_uuid': {'name': 'skill',
   'restrictions': [{'issuer_did': 'PQRXDxdGqQGSZ8z69p4xZP'}],
   'non_revoked': {'to': 1672627475}}},
 'requested_predicates': {},
 'non_revoked': {'to': 1672627476},
 'nonce': '174512802480630886890112'}

In [23]:
if state == "request_received":
    print(
    "Received Request -> Query for credentials in the wallet that satisfy the proof request")
    
if presentation_request != None:
    # include self-attested attributes (not included in credentials)
    credentials_by_reft = {}
    revealed = {}
    self_attested = {}
    predicates = {}

    # select credentials to provide for the proof
    credentials = await agent_controller.proofs.get_presentation_credentials(pres_ex_id)
    credentials = credentials[1:]
    print(credentials)

    if credentials:
        for row in sorted(
            credentials,
            key=lambda c: dict(c["cred_info"]["attrs"]),
            reverse=True,
        ):
            for referent in row["presentation_referents"]:
                if referent not in credentials_by_reft:
                    credentials_by_reft[referent] = row

    for referent in presentation_request["requested_attributes"]:
        if referent in credentials_by_reft:
            revealed[referent] = {
                "cred_id": credentials_by_reft[referent]["cred_info"][
                    "referent"
                ],
                "revealed": True,
            }

        else:
            self_attested[referent] = "South Africa"

    for referent in presentation_request["requested_predicates"]:
        if referent in credentials_by_reft:
            predicates[referent] = {
                "cred_id": credentials_by_reft[referent]["cred_info"][
                    "referent"
                ]
            }

    print("\nGenerate the proof")
    proof = {
        "requested_predicates": predicates,
        "requested_attributes": revealed,
        "self_attested_attributes": self_attested,
    }
    print(proof)
    print("\nXXX")
    print("\npredicates:\n{}".format(predicates))
    print("\nrevealed:\n{}".format(revealed))
    print("\nself_attested:\n{}".format(self_attested))
    
else: 
    print("No presenation record identifier")

[{'cred_info': {'referent': 'My Revocable Credential', 'attrs': {'name': 'Bob', 'age': '21', 'skill': 'researcher'}, 'schema_id': 'PQRXDxdGqQGSZ8z69p4xZP:2:test_revocable_schema:0.0.1', 'cred_def_id': 'PQRXDxdGqQGSZ8z69p4xZP:3:CL:20:default', 'rev_reg_id': 'PQRXDxdGqQGSZ8z69p4xZP:4:PQRXDxdGqQGSZ8z69p4xZP:3:CL:20:default:CL_ACCUM:f0bdd25d-4fe5-4edc-858a-293a6aa2cb54', 'cred_rev_id': '1'}, 'interval': {'from': None, 'to': 1672627475}, 'presentation_referents': ['0_skill_uuid']}]

Generate the proof
{'requested_predicates': {}, 'requested_attributes': {'0_skill_uuid': {'cred_id': 'My Revocable Credential', 'revealed': True}}, 'self_attested_attributes': {}}

XXX

predicates:
{}

revealed:
{'0_skill_uuid': {'cred_id': 'My Revocable Credential', 'revealed': True}}

self_attested:
{}


## 22. Send presentation

Bob sends the proof presentation to Alice for verification.

In [24]:
response = await agent_controller.proofs.send_presentation(pres_ex_id, proof)
print(response)

{'trace': False, 'thread_id': '49315c35-3a9f-4218-a647-6e71319578f1', 'role': 'prover', 'presentation_request_dict': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/present-proof/1.0/request-presentation', '@id': '49315c35-3a9f-4218-a647-6e71319578f1', 'request_presentations~attach': [{'@id': 'libindy-request-presentation-0', 'mime-type': 'application/json', 'data': {'base64': 'eyJuYW1lIjogIlByb29mIG9mIFBlcnNvbmFsIEluZm9ybWF0aW9uIiwgInZlcnNpb24iOiAiMS4wIiwgInJlcXVlc3RlZF9hdHRyaWJ1dGVzIjogeyIwX3NraWxsX3V1aWQiOiB7Im5hbWUiOiAic2tpbGwiLCAicmVzdHJpY3Rpb25zIjogW3siaXNzdWVyX2RpZCI6ICJQUVJYRHhkR3FRR1NaOHo2OXA0eFpQIn1dLCAibm9uX3Jldm9rZWQiOiB7InRvIjogMTY3MjYyNzQ3NX19fSwgInJlcXVlc3RlZF9wcmVkaWNhdGVzIjoge30sICJub25fcmV2b2tlZCI6IHsidG8iOiAxNjcyNjI3NDc2fSwgIm5vbmNlIjogIjE3NDUxMjgwMjQ4MDYzMDg4Njg5MDExMiJ9'}}]}, 'connection_id': '9777a4c5-00c9-4816-acd3-3139431cee8b', 'updated_at': '2023-01-02 02:45:16.183365Z', 'initiator': 'external', 'state': 'presentation_sent', 'presentation_exchange_id': '3185a03

## 23. Continue in [Alice's Notebook](http://localhost:8888/lab/tree/5%20Advanced%20Concepts/Part%202%20-%20Revocation.ipynb)

Alice will verify the now revoked credential to be NOT OK.

## End of Tutorial

Be sure to terminate the controller.

In [25]:
response = await agent_controller.terminate()
print(response)

None
